In [29]:
# Necessary imports
import torch
import torch.nn as nn
import torch.nn.functional as F 
import torch.optim as optim
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, random_split
from torchvision.datasets import ImageFolder
from torchvision.models import detection
from PIL import Image
import os
import glob
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle
import seaborn as sns
import numpy as np
from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn.utils.class_weight import compute_class_weight


import train
import models

# Check for device: use MPS if available, otherwise use CPU
device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")
print(f"Using device: {device}")

# Define dataset path
# dataset_path = '/Users/enxom/Desktop/INM705 CW/dataset1'
dataset_path = './dataset1'

# Collect data helper functions
def is_valid_file(file_path):
    valid_extensions = ['.jpg', '.jpeg', '.png', '.ppm', '.bmp', '.pgm', '.tif', '.tiff', '.webp']
    return any(file_path.endswith(ext) for ext in valid_extensions)

def collect_data(directory):
    data = []
    labels = []
    class_to_idx = {cls: idx for idx, cls in enumerate(os.listdir(directory)) if os.path.isdir(os.path.join(directory, cls))}
    for cls, idx in class_to_idx.items():
        class_path = os.path.join(directory, cls)
        for file_path in glob.glob(os.path.join(class_path, '*')):
            if is_valid_file(file_path):
                data.append(file_path)
                labels.append(idx)
    return data, labels, class_to_idx



# Data transformations and dataset preparation
transform = transforms.Compose([
    transforms.Resize((224, 224)), 
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor()
])

# Custom Dataset Class
class CustomImageDataset(torch.utils.data.Dataset):
    def __init__(self, data, labels, transform=None):
        self.data = data
        self.labels = labels
        self.transform = transform

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        image_path = self.data[idx]
        label = self.labels[idx]
        image = Image.open(image_path).convert('RGB')
        if self.transform:
            image = self.transform(image)
        return image, label

# Load and split dataset
data, labels, class_to_idx = collect_data(dataset_path)
dataset = CustomImageDataset(data, labels, transform)
train_size = int(0.7 * len(dataset))
val_size = int(0.15 * len(dataset))
test_size = len(dataset) - train_size - val_size
train_dataset, val_dataset, test_dataset = random_split(dataset, [train_size, val_size, test_size])
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

# Compute class weights for imbalanced datasets
class_weights = compute_class_weight('balanced', classes=np.unique(labels), y=labels)
torch.save(class_weights, 'checkpoints/waste.pth')
class_weights_tensor = torch.tensor(class_weights, dtype=torch.float32, device=device)

class_weights_dict = dict(zip(np.unique(labels), class_weights))
def run_faster_cnn(img, model, classes):
    img = np.array(Image.open(img))
    img = transforms.toTensor()(img)
    out = model([img])
    scores = out[0]['scores']
    boxes = out[0]['boxes']
    classes = out[0]['classes']
    if len(scores) > 0:
        plt.imshow(img)
        ax = plt.gca()
        for box in boxes.detach().numpy():
            xbox_min, ybox_min, xbox_max, ybox_max = box
            width = xbox_max - xbox_min
            height = ybox_max - ybox_min
            rect = Rectangle((xbox_min, ybox_min), width, height, linewidth=2, edgecolour='r', facecolor='none')
            ax.add_patch(rect)
        plt.show()
    class_labels = [classes[val.item()-1] for val in classes]
    return class_labels

# faster_weights = torch.load("checkpoints/waste.pth", map_location=device)
print(class_weights_tensor.size())
# faster_weights = class_weights_tensor
# print(type(detection.FasterRCNN_ResNet50_FPN_Weights))
# print(type(class_weights_tensor))
# print(type(faster_weights))

def load_model(weights_dict):
    # frcnn_args_inference = {'box_score_thresh': 0.75, 'box_detections_per_img': 32}
    model = detection.fasterrcnn_resnet50_fpn(weights_dict)
    # model = detection.fasterrcnn_resnet50_fpn(pretrained=True)
    # model.load_state_dict(weights_dict)
    model.eval()

    # # Load the pre-trained Faster R-CNN model
    # model = detection.fasterrcnn_resnet50_fpn(pretrained=True)

    # # # Load your custom weights (assuming weights_tensor is your PyTorch tensor with size 6, each being 224x224)
    # # weights_tensor = ...  # Your PyTorch tensor weights here

    # # Define component names where you want to load the weights
    # component_names = ['backbone', 'rpn', 'roi_heads']

    # # Load weights into specific components of the model
    # for name, weights in zip(component_names, weights_tensor):
    #     component = getattr(model, name)
    #     component.load_state_dict({'weight': weights})

    # # Ensure the model is in evaluation mode after loading weights
    # model.eval()


    return model

faster_cnn_model = load_model(class_weights_dict)
faster_cnn_model._modules.keys()

def resnet18(num_classes):
    return models.ResNet(models.BasicBlock, [2, 2, 2, 2], num_classes)

# Instantiate both models
num_classes=len(class_to_idx)
# model1 = models.WasteClassifier(num_classes).to(device)
# model2 = models.resnet18(num_classes).to(device)
model3 = faster_cnn_model.to(device)

# Optimizers and loss functions
# optimizer1 = optim.Adam(model1.parameters(), lr=0.001)
# optimizer2 = optim.Adam(model2.parameters(), lr=0.001)
optimizer3 = optim.Adam(model3.parameters(), lr=0.001)
# criterion1 = nn.CrossEntropyLoss(weight=class_weights_tensor)
# criterion2 = nn.CrossEntropyLoss(weight=class_weights_tensor)
criterion3 = nn.CrossEntropyLoss(weight=class_weights_tensor)


# Train both models separately
# print("\nTraining WasteClassifier...")
# train.train_model(model1, optimizer1, criterion1, train_loader, val_loader)

# print("\nTraining Custom ResNet Model...")
# train.train_model(model2, optimizer2, criterion2, train_loader, val_loader)

print("\nTraining FasterCNN...")
# train.train_model(model3, optimizer3, criterion3, train_loader, val_loader)
# How to call it?

# Evaluation function
def evaluate_model(model, test_loader):
    model.eval()
    total_test = 0
    correct_test = 0
    all_labels = []
    all_preds = []

    with torch.no_grad():
        for inputs, labels in test_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)
            total_test += labels.size(0)
            correct_test += (preds == labels).sum().item()
            all_labels.extend(labels.cpu().numpy())
            all_preds.extend(preds.cpu().numpy())

    accuracy = correct_test / total_test * 100
    print(f'Test Accuracy: {accuracy:.2f}%')

    # Compute confusion matrix
    cm = confusion_matrix(all_labels, all_preds)
    plt.figure(figsize=(10, 7))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
    plt.xlabel('Predicted Labels')
    plt.ylabel('True Labels')
    plt.title('Confusion Matrix')
    plt.show()

# Evaluate both models
print("\nEvaluating WasteClassifier...")
# evaluate_model(model1, test_loader)

print("\nEvaluating Custom ResNet Model...")
# evaluate_model(model2, test_loader)

print("\nEvaluating Faster CNN Model...")
evaluate_model(model3, test_loader)


Using device: cpu
torch.Size([6])


/home/milan/INM705-Deep-Learning-for-Image-Analysis/venv/lib/python3.11/site-packages/torchvision/models/_utils.py:135: UserWarning: Using 'weights' as positional parameter(s) is deprecated since 0.13 and may be removed in the future. Please use keyword parameter(s) instead.
  warnings.warn(
/home/milan/INM705-Deep-Learning-for-Image-Analysis/venv/lib/python3.11/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=FasterRCNN_ResNet50_FPN_Weights.COCO_V1`. You can also use `weights=FasterRCNN_ResNet50_FPN_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)



Training FasterCNN...


AssertionError: targets should not be none when in training mode